<img src=https://www.nticmaster.es/wp-content/uploads/2021/03/logotipos-ucm-ntic.png>

<img src=https://www.luismaram.com/wp-content/uploads/2008/07/walmart_logo.gif>

# 1. Introducción

Las cadenas de Retail siempre han trabajado en un entorno competitivo con márgenes cada vez más ajustados. El correcto uso de los datos puede marcar la diferencia, por tanto predecir las ventas así como conocer la eficacia en los diferentes  departamentos, las campañas comerciales y el comportamiento del consumidor puede marcar la diferencia y nos ayudara a ser comercialmente más eficientes, tener un correcto stock de los productos, ajustar la plantilla de trabajo entre otros. 
El presente trabajo permite facilitar y automatizar esta tarea con el fin de facilitar la toma de decisiones.
Para realizar este trabajo se han utilizado datos históricos de ventas para 45 tiendas Walmart ubicadas en diferentes regiones. Cada tienda contiene varios departamentos y se tiene la tarea de predecir las ventas de todo el departamento para cada tienda.
Además, Walmart organiza varios eventos promocionales de rebajas durante todo el año. Estas rebajas preceden a los feriados importantes, los cuatro más grandes de los cuales son el Super Bowl, el Labor Day, Thanksgiving y Christmas. Las semanas que incluyen estos días festivos se ponderan cinco veces más en la evaluación que las semanas que no son festivos. Parte del desafío que presenta esta competencia es modelar los efectos de las rebajas en estas semanas de vacaciones en ausencia de datos históricos completos / ideales.
De forma adicional se estudia el comportamiento de las ventas en las tiendas según su formato y conocer cuales son los departamentos que mas contribuyen a la cifra semanal. Se estudiara la estacionalidad de las ventas y los resultados de los cinco tipos de rebajas a lo largo del año para conocer como de efectivas ha sido tanto de forma individual y conjunta.
Analizaremos el comportamiento del consumidor en función de si la semana es festiva o no, la temperatura ambiente, el precio del carburante, el IPC así como la tasa de desempleo.

#### El presente estudio forma parte del proceso de evaluacion del master de Big Data & Data Science impartido por la Universidad Complutense de Madrid y NTIC



# 1. Abstract


Retail chains have always worked in a competitive environment with increasingly tight margins. The correct use of data can make a difference, therefore predicting sales as well as knowing the effectiveness in different departments, commercial campaigns and consumer behavior can make a difference and help us to be more commercially efficient, have a correct Stock of products, adjust the workforce among others.
The present work allows to facilitate and automate this task in order to facilitate decision-making.
To carry out this work, historical sales data have been used for 45 Walmart stores located in different regions. Each store contains several departments and you are tasked with predicting the sales of the entire department for each store.
In addition, Walmart organizes several promotional sales events throughout the year. These sales precede major holidays, the four biggest of which are the Super Bowl, Labor Day, Thanksgiving and Christmas. The weeks that include these holidays are weighted five times more in the evaluation than the weeks that are not holidays. Part of the challenge this competition presents is modeling the effects of sales on these vacation weeks in the absence of complete / ideal historical data.
Additionally, the behavior of sales in stores is studied according to their format and to know which are the departments that contribute the most to the weekly figure. The seasonality of sales and the results of the five types of sales throughout the year will be studied to know how effective it has been both individually and jointly.
We will analyze consumer behavior depending on whether the week is a holiday or not, the ambient temperature, the price of fuel, the CPI as well as the unemployment rate.

#### This study is part of the evaluation process of the Master of Big Data & Data Science taught by the Complutense University of Madrid and NTIC


## 1. Importar Librerías & Cargar datos

<img src=https://image.shutterstock.com/image-photo/download-data-storage-business-technology-600w-1420922999.jpg>

#### ¿Qué haremos en esta sección?

* En esta sección importaremos las librerías necesarias y los datos desde kaggle.com
* Haremos una inspeccion inicial de los archivos
* Procederemos a crear los dataframes mergeandolos unos con otros
* Convertiremos la columna "Date" a formato datetime
* Descompondremos la fecha creando nuevas variables 'Day','Week','Month','Year

###  1.1 Librerías a importar

In [ ]:
!pip install numpy pandas matplotlib seaborn --quiet


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')
from sklearn import metrics

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from datetime import datetime
import math
import plotly.express as px
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')

!pip install pingouin
import pingouin as pg

 


### 1.2 Carga de datos

In [ ]:
features = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
stores = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv')
train = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
test = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
sampleSubmission = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')

### 1.3 Inspección inicial
Observamos el tipo de dato de cada una de las variables para los diferentes archivos:

In [ ]:
pd.DataFrame(features.dtypes, columns=['Type']).T

In [ ]:
pd.DataFrame(stores.dtypes, columns=['Type']).T

In [ ]:
pd.DataFrame(train.dtypes, columns=['Type']).T

In [ ]:
pd.DataFrame(test.dtypes, columns=['Type']).T

Evaluamos la cantidad de filas y columnas en cada uno de los datasets. Observamos que train y test comprenden el 78% y 22% respectivamente del total de los datos de las 45 tiendas.

In [ ]:
print(features.shape)
print(stores.shape)
print(train.shape)
print(test.shape)
print("El ratio de train data : test data es ", 
      (round(train.shape[0]*100/(train.shape[0]+test.shape[0])),100-round(train.shape[0]*100/(train.shape[0]+test.shape[0]))))

¿Qué información hemos obtenido hasta ahora?

* El dataset _'features', 'train' y 'test'_ poseen la fecha en formato objet y debe ser modificada.
* El dataset 'stores' posee 45 filas correspondientes al número de tiendas que vamos a analizar
* Los conjuntos _'train'_ y _'test_ comprenden el 78% y 22% respectivamente del total de los datos de estudio.

###  1.4 Merge de datasets 'features' y 'stores'

Vamos a crear un nuevo dataset en el que a 'Features' le vamos a incluir el área de la tienda, así como su tipo que 
aparecen en 'Stores' utilizando como elemento común entre ambos la columna 'Store' siendo la unión a través de inner join.
Comprobamos que el nuevo dataset "feature_store" posee las 8190 filas, pero con 14 variables en vez de 12 para las 45 tiendas.




In [ ]:
feature_store = features.merge(stores, how='inner', on = "Store")

In [ ]:
print("El numero de filas y columnas es de: ",(feature_store.shape))
print("Existen",(len(feature_store.Store.unique())), 'tiendas unicas')

### 1.5 Merge de datasets 'train' & 'feature_store'

Realizamos la operación para los dataset 'train' & 'feature_store'.

In [ ]:

train_df = train.merge(feature_store, how='inner', 
                       on = ['Store','Date','IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
print("El numero de filas y columnas es de: ",(train_df.shape))

### 1.6 Merge de datasets 'test' & 'feature_store'

Realizamos la misma operación para los dataset 'test' & 'feature_store'. El nuevo dataset mantiene las mismas filas
que 'test' y tiene las mismas variables que 'train’ a excepción de 'Weekly_Sales'

In [ ]:
test_df = test.merge(feature_store, how='inner', 
                     on = ['Store','Date','IsHoliday']).sort_values(by = ['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
print("El numero de filas y columnas es de: ",(test_df.shape))

### 1.7 Convertir columna "Date" a formato datetime

In [ ]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])

print(train_df["Date"].dtypes)
print(test_df["Date"].dtypes)

### 1.8 Descomponemos la fecha creando nuevas variables 'Day','Week','Month','Year'

In [ ]:
train_df['Day'] = train_df['Date'].dt.day
train_df['Week'] = train_df['Date'].dt.week
train_df['Month'] = train_df['Date'].dt.month
train_df['Year'] = train_df['Date'].dt.year


test_df['Day'] = test_df['Date'].dt.day
test_df['Week'] = test_df['Date'].dt.week
test_df['Month'] = test_df['Date'].dt.month
test_df['Year'] = test_df['Date'].dt.year

print("El numero de filas y columnas es de: ",(train_df.shape))
print("El numero de filas y columnas es de: ",(test_df.shape))


In [ ]:
train_df.head(1)

In [ ]:
test_df.head(1)

¿Que hemos conseguido?

* Hemos obtenido dos dataframes _'train_df' y _'test_df’ donde hemos mergeado en 'train y 'test' las columnas de los dataframes de 'features' y 'stores'. 
* La columna 'Date' estaba en formato objeto y se ha cambiado al formato date.
* Hemos descompuesto la variable 'Date' creando nuevas variables 'Day','Week', 'Month' y 'Year'.


## 2.  Exploratory Data Analysis

# <img src=https://image.shutterstock.com/image-vector/big-data-visualization-abstract-graphic-600w-1106564813.jpg>

#### Que haremos en esta sección?

* Analizaremos los diferentes formatos de tienda.
* Obtendremos un boxplot para cada superficie y formato de tienda.
* Grafico de barras de barras de superficie para cada determinante de tienda.
* Grafico de ventas semanales por formato de tienda.
* Promedio de ventas mensuales por formato de tienda.
* Promedio de ventas mensuales para cada año.
* Promedio de ventas para cada día del mes.
* Promedio de ventas semanales a través de una línea temporal.
* Ventas semanales por departamento.
* Análisis de ventas de las 52 semanas del año.
* Análisis de los resultados de las Rebajas 1.
* Análisis de los resultados de las Rebajas 2.
* Análisis de los resultados de las Rebajas 3.
* Análisis de los resultados de las Rebajas 4.
* Análisis de los resultados de las Rebajas 5.
* Análisis de los resultados de las Rebajas Totales.
* Análisis de ventas si la semana es festiva.
* Contraste entre ventas semanales y temperatura ambiente
* Contraste entre ventas semanales y precio de carburante
* Contraste entre ventas semanales e IPC.
* Contraste entre ventas semanales y desempleo.
* Matriz de correlación.
* Tabla de correlación de las diferentes variables vs Ventas Semanales


### 2.1 Análisis de formatos de tiendas y ventas

#### 2.1.1 Análisis de los formatos de tienda

Observamos que el formato dominante son las tiendas de tipo A seguidos del formato B y C respectivamente.

In [ ]:
ax = sns.countplot(stores.Type ,facecolor=(0,0,0,0),linewidth=10,
                   edgecolor=sns.color_palette("spring", 3))
for p in ax.patches:
    ax.annotate(f'Number of\n stores:\n {p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()-4),
               ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points',fontsize=12);

#### 2.1.2 Boxplot Superficie y formato de tienda
Para los tres formatos de tienda, el modelo A es el mayor tamaño. El formato B es el modelo que presenta un rango de superficie mayor. finalmente, el modelo de tienda C posee un formato que con una superficie similar a todas las tiendas siendo las tiendas de tipo C a un formato de tienda de conveniencia.

In [ ]:
store_type = pd.concat([stores['Type'], stores['Size']], axis=1)
plt.figure(figsize=(15, 6))
sns.boxplot(x='Type', y='Size', data=store_type);

#### 2.1.3 Grafico de barras Superficie y Tienda

Complementando al boxplot del apartado 2.2, se observa un primer bloque de 10 tiendas con superficie similar, esto es debido a los outliers del formato B que se solapan con la superficie del formato A.

Posteriormente se observa un grupo intermedio de tiendas con una superficie bastante variable que corresponden al formato B y A.

En el último tramo existen tiendas con una superficie superior a 200,000 que pertenecen solamente al grupo A.


In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x='Store',y='Size',data=stores,order=stores.sort_values('Size')['Store'].tolist())
plt.title('Superficie de todas las tiendas.',fontsize=15)
plt.tight_layout();


#### 2.1.4 Grafico ventas semanales por formato de tienda

In [ ]:
print('Type vs Weekly_Sales')
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.stripplot(y=train_df['Weekly_Sales'],x=train_df['Type'])
plt.subplot(1,2,2)
sns.boxenplot(y=train_df['Weekly_Sales'],x=train_df['Type']);

#### 2.1.5 Promedio de ventas mensuales por formato de tienda

El mayor promedio de ventas mensuales se produce en el formato A debido a que existen más unidades y su formato es mayor.
El formato B posee un comportamiento similar al A con una curva en el crecimiento de las ventas a partir del mes 10.
Para el formato C se aprecia un comportamiento lineal con una leve caída en ventas el mes 12, esto podría ser debido a que las tiendas de menor tamaño ofrecen un servicio de tiendas de proximidad donde abastece de los productos de uso diario y el mes de diciembre al aumentarse las ventas en los formatos A y B, cubre las necesidades de las familias no siendo necesaria la compra en el formato C. Se observa por tanto una correlación negativa entre los formatos A y C, así como B y C.


In [ ]:
plt.figure(figsize=(15,3))
train_df[train_df['Type']=='A'].groupby('Month').mean()['Weekly_Sales'].plot()
train_df[train_df['Type']=='B'].groupby('Month').mean()['Weekly_Sales'].plot()
train_df[train_df['Type']=='C'].groupby('Month').mean()['Weekly_Sales'].plot()
plt.title('Promedio de ventas mensuales de por formato de tienda', fontsize=18)
plt.legend(['Type A', 'Type B', 'Type C'], loc='best', fontsize=16)
plt.ylabel('Ventas', fontsize=16)
plt.xlabel('Meses', fontsize=16);

In [ ]:
print("Correlacion del promedio de ventas mensuales de los formatos A y C")
display(pg.corr(train_df['Type']=='A', train_df['Type']=='C', method='pearson'))
print("--------------------------------------------------------")
print("Correlacion del promedio de ventas mensuales de los formatos B y C")
display(pg.corr(train_df['Type']=='B', train_df['Type']=='C', method='pearson'))

#### 2.1.6 Promedio de ventas mensuales por año.

Se observa un comportamiento similar en casi todos los meses para los años 2010,2011 y 2012. Las mayores ventas se producen en los meses 11 y 12 y tras las Navidades las ventas llegan a sus valores mínimos

In [ ]:
for df in [train_df,test_df]:
    df['Week'] = df['Date'].dt.week
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year

plt.figure(figsize=(15,3))
train_df[train_df['Year']==2010].groupby('Month').mean()['Weekly_Sales'].plot()
train_df[train_df['Year']==2011].groupby('Month').mean()['Weekly_Sales'].plot()
train_df[train_df['Year']==2012].groupby('Month').mean()['Weekly_Sales'].plot()
plt.title('Promedio de ventas mensuales de Walmart para cada año', fontsize=18)
plt.legend(['2010', '2011', '2012'], loc='best', fontsize=16)
plt.ylabel('Ventas', fontsize=16)
plt.xlabel('Meses', fontsize=16);


#### 2.1.7 Promedio de ventas diarias para cada año.

Las ventas diarias se mantienen de forma relativamente constante en el mes. Los picos que se observan los días 23, 24 y 25 son motivados por el aumento de ventas en los meses de noviembre y diciembre.


In [ ]:
for df in [train_df,test_df]:
    df['Week'] = df['Date'].dt.week
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year

plt.figure(figsize=(15,3))
train_df[train_df['Year']==2010].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Year']==2011].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Year']==2012].groupby('Day').mean()['Weekly_Sales'].plot()
plt.title('Promedio de ventas para cada dia del mes', fontsize=18)
plt.legend(['2010', '2011', '2012'], loc='best', fontsize=16)
plt.ylabel('Ventas', fontsize=16)
plt.xlabel('Dias', fontsize=16);

#### 2.1.8 Promedio de ventas diarias por meses

Las ventas sufren un ligero descenso según avanzan los días del mes a excepción de los meses de noviembre y diciembre.


In [ ]:
for df in [train_df,test_df]:
    df['Week'] = df['Date'].dt.week
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year

plt.figure(figsize=(15,8))
train_df[train_df['Month']==1].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==2].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==3].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==4].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==5].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==6].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==7].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==8].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==9].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==10].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==11].groupby('Day').mean()['Weekly_Sales'].plot()
train_df[train_df['Month']==12].groupby('Day').mean()['Weekly_Sales'].plot()
plt.title('Promedio de ventas de walmart para cada dia del mes', fontsize=18)
plt.legend(['Ene', 'Feb', 'Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic'], loc='best', fontsize=16)
plt.ylabel('Ventas', fontsize=16)
plt.xlabel('Dias', fontsize=16);

#### 2.1.9 Línea temporal con el promedio de las ventas semanales.

Se observa un patrón que se repite de forma similar para los años de estudio.


In [ ]:
plt.figure(figsize=(15,3))
train_df.groupby('Date')['Weekly_Sales'].mean().plot()
plt.title('Promedio de las ventas semanales de Walmart a traves de una linea de tiempo', fontsize=18)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Date', fontsize=16);

In [ ]:
data = pd.concat([train_df['Week'], train_df['Weekly_Sales']], axis=1)
f, ax = plt.subplots(figsize=(20, 6))
fig = sns.boxplot(x='Week', y="Weekly_Sales", data=data, showfliers=False)

#### 2.1.10 Ventas Semanales y Departamento

En la siguiente grafica de barras se comprueba que no todos los departamentos contribuyen de la misma manera en las ventas semanales

In [ ]:
plt.figure (figsize = (20,7))
sns.set_style('darkgrid')
sns.barplot ( data = train_df, x = 'Dept', y =  'Weekly_Sales');

#### 2.1.11 Análisis de ventas de las 52 semanas

Se observa durante las cuatro primeras semanas de enero el periodo de ventas más bajo del año, con una fuerte subida en febrero donde se mantendrían las cifras en unos valores relativamente constantes hasta la semana 43. Entre la semana 44 hasta las 49 se
producen unas ventas muy bajas a excepción de la semana 47 donde se produce una fuerte subida en las ventas.

La semana 51 comprende las fechas entre el 20 y el 26 de diciembre estando por tanto en Navidades, es comprensible que se produzcan en esa semana las mayores ventas del año.

Se comprueba que las ventas tienen un componente estacionario a lo largo de los años al observarse que repite el mismo patrón en 2010, 2011 y 2012.

In [ ]:
df_weeks = train_df.groupby('Week').sum()

In [ ]:
weekly_sales = train_df.groupby(['Year','Week'], as_index = False).agg({'Weekly_Sales': ['mean', 'median']})
weekly_sales2010 = train_df.loc[train_df['Year']==2010].groupby(['Week']).agg({'Weekly_Sales': ['mean', 'median']})
weekly_sales2011 = train_df.loc[train_df['Year']==2011].groupby(['Week']).agg({'Weekly_Sales': ['mean', 'median']})
weekly_sales2012 = train_df.loc[train_df['Year']==2012].groupby(['Week']).agg({'Weekly_Sales': ['mean', 'median']})

In [ ]:
fig7 = go.Figure()
fig7.add_trace(go.Scatter( x = weekly_sales2010['Weekly_Sales']['mean'].index, y = weekly_sales2010['Weekly_Sales']['mean'], name = 'Media Ventas 2010', mode = 'lines') )
fig7.add_trace(go.Scatter( x = weekly_sales2011['Weekly_Sales']['mean'].index, y = weekly_sales2011['Weekly_Sales']['mean'], name = 'Media Ventas 2011', mode = 'lines') )
fig7.add_trace(go.Scatter( x = weekly_sales2012['Weekly_Sales']['mean'].index, y = weekly_sales2012['Weekly_Sales']['mean'], name = 'Media Ventas 2012', mode = 'lines') )
fig7.update_layout(title = 'Ventas 2010, 2011, 2012', xaxis_title = 'Semanas')


### 2.2 Análisis de los resultados de las Rebajas

#### 2.2.1 Análisis de ventas de las 52 semanas vs Rebajas1

Analizando el grafico resultante se observa una fuerte subida en las ventas en la semana 5 y en Rebajas1 (MarkDown1).
Existe una fuerte correlación del 99% entre las cinco primeras semanas del año y la estrategia de ventas con las rebajas Markdown1, por tanto, se supone que pudiera ser una estrategia comercial enfocada en superar la cuesta de enero.


In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown1'], name = 'MarkDown1', mode = 'lines') )
fig1.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['Weekly_Sales'], name = 'Ventas semanales', mode = 'lines') )
fig1.update_layout(title = 'Ventas vs Markdown1', xaxis_title = 'Semanas')

In [ ]:
df_weeks_Markdown1 = df_weeks.iloc[[0,1,2,3,4]]
print("Correlacion entre Markdow1 y las cinco primeras semanas del anno")
display(pg.corr(df_weeks_Markdown1['Weekly_Sales'], df_weeks_Markdown1['MarkDown1'], method='pearson'))

#### 2.2.2 Análisis de ventas de las 52 semanas vs Rebajas2

Analizando el comportamiento de las Rebajas2 (Markdown2), se aprecia una estrategia para fechas especificas (semanas desde la 01 hasta la 08, semana 14, semana 45 y semana 52). La correlación es moderadamente negativa del -0.44. Por tanto, al aumentar las rebajas disminuyen las ventas, esto podría ser debido a que se trata de productos básicos donde las unidades vendidas son las mismas independientemente del descuento y el aplicar las rebajas existe menos facturación.


In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown2'], name = 'MarkDownw2', mode = 'lines') )
fig2.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['Weekly_Sales'], name = 'Ventas semanales', mode = 'lines') )
fig2.update_layout(title = 'Ventas vs Markdown2', xaxis_title = 'Semanas')

In [ ]:
df_weeks_Markdown2 = df_weeks.iloc[[0,1,2,3,4,5,6,7,8,13,44,51]]
print("Correlacion entre las semanas de promocion Markdown2 y las ventas semanales")
display(pg.corr(df_weeks_Markdown2['Weekly_Sales'], df_weeks_Markdown2['MarkDown2'], method='pearson'))

#### 2.2.3 Análisis de ventas de las 52 semanas vs Rebajas3
Se trataría de una promoción específica para la semana 47 del día de Acción de Gracias (25 de noviembre) con una correlación positiva del 99%


In [ ]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown3'], name = 'MarkDown3', mode = 'lines') )
fig3.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['Weekly_Sales'], name = 'Ventas semanales', mode = 'lines') )
fig3.update_layout(title = 'Ventas vs Markdown3', xaxis_title = 'Semanas')

In [ ]:
df_weeks_Markdown3 = df_weeks.iloc[[45,46,47]]
print("Correlacion entre las semanas de promocion Markdown3 y las ventas semanales ")
display(pg.corr(df_weeks_Markdown3['Weekly_Sales'], df_weeks_Markdown3['MarkDown3'], method='pearson'))

#### 2.2.4 Análisis de ventas de las 52 semanas vs Rebajas4

Es una campaña enfocada en las cuatro primeras semanas del año, la semana 9 y la semana 31 con una fuerte correlación positiva del 82%.


In [ ]:
fig4 = go.Figure()
fig4.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown4'], name = 'MarkDown4', mode = 'lines') )
fig4.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['Weekly_Sales'], name = 'Ventas semanales', mode = 'lines') )
fig4.update_layout(title = 'Ventas vs Markdown4', xaxis_title = 'Semanas')

In [ ]:
df_weeks_Markdown4 = df_weeks.iloc[[0,1,2,3,4,8,30]]
print("Correlacion entre las semanas de promocion Markdown4 y las ventas semanales ")
display(pg.corr(df_weeks_Markdown4['Weekly_Sales'], df_weeks_Markdown4['MarkDown4'], method='pearson'))

#### 2.2.5 Análisis de ventas de las 52 semanas vs Rebajas5

Se produce un incremento del modelo de rebajas Markdown5 en la semana 48 siendo esta la semana posterior al Dia de Acción de Gracias y su finalidad podría ser eliminar los posibles excedentes de aquellos productos enfocados al 25 de noviembre o bien intentar evitar una caída en las ventas en la semana posterior a la festividad. La correlación es moderadamente negativa del -54% sugiriendo que se está eliminando stock en posibles productos perecederos.



In [ ]:
fig5 = go.Figure()
fig5.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown5'], name = 'MarkDown5', mode = 'lines') )
fig5.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['Weekly_Sales'], name = 'Ventas semanales', mode = 'lines') )
fig5.update_layout(title = 'Ventas vs Markdown5', xaxis_title = 'Semanas')

In [ ]:
df_weeks_Markdown5 = df_weeks.iloc[[45,46,47]]
print("Correlacion entre las semanas de promocion 46,47 y 48 Markdown5 y las ventas semanales ")
display(pg.corr(df_weeks_Markdown5['Weekly_Sales'], df_weeks_Markdown5['MarkDown5'], method='pearson'))

#### 2.2.6 Análisis de ventas de las 52 semanas vs Rebajas Totales

El éxito en las diferentes campañas permite obtener una linealidad en las ventas a lo largo del año. No obstante, existen periodos concretos donde se produce un significativo descenso en las ventas, estos serían las cuatro primeras semanas del año y la semana anterior y posterior al Dia de Acción de Gracias.


In [ ]:
fig6 = go.Figure()
fig6.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown1'], name = 'MarkDown1', mode = 'lines') )
fig6.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown2'], name = 'MarkDown2', mode = 'lines') )
fig6.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown3'], name = 'MarkDown3', mode = 'lines') )
fig6.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown4'], name = 'MarkDown4', mode = 'lines') )
fig6.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['MarkDown5'], name = 'MarkDown5', mode = 'lines') )
fig6.add_trace(go.Scatter( x = df_weeks.index, y = df_weeks['Weekly_Sales'], name = 'Weekly Sales', mode = 'lines+markers') )
fig6.update_layout(title = 'Ventas vs Total Markdown', xaxis_title = 'Semanas')


### 2.3 Análisis de diferentes variables


#### 2.3.1 Análisis de ventas si la semana es festiva

Cuando la semana es festiva se observa un aumento de las ventas semanales con bastante dispersión de valores frente a una semana regular.


In [ ]:
print('IsHoliday vs Weekly_Sales')
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.stripplot(y=train_df['Weekly_Sales'],x=train_df['IsHoliday'])
plt.subplot(1,2,2)
sns.violinplot(y=train_df['Weekly_Sales'],x=train_df['IsHoliday']);

#### 2.3.2 Contraste entre Ventas Semanales y temperatura ambiente

Se observa que la las ventas se mantienen estables independientemente de la temperatura exterior como norma general.




In [ ]:
# Convertir la temperatura a grados centigrados para una mejor interpretacion.
#train_df['Temperature'] = train_df['Temperature'].apply(lambda x :  (x - 32) / 1.8)
#train_df['Temperature'] = train_df['Temperature'].apply(lambda x :  (x - 32) / 1.8 )
train_df['Temperature'] = (train_df['Temperature']-32.)/1.8
test_df['Temperature'] = (test_df['Temperature']-32.)/1.8

In [ ]:
plt.figure (figsize = (15,7))
sns.set_style('darkgrid')
sns.scatterplot ( data = train_df, x = 'Temperature', y =  'Weekly_Sales', hue = 'IsHoliday');


In [ ]:
plt.figure(figsize=(15,7))
sns.scatterplot(x=train_df.Temperature, y=train_df.Weekly_Sales, hue=train_df.Type, s=80);

#plt.xticks( fontsize=16)
#plt.yticks( fontsize=16)
sns.set_style('darkgrid')
plt.xlabel('Temperature', fontsize=18)
plt.ylabel('Sales', fontsize=18);

#### 2.3.3 Contraste entre Ventas Semanales y precio de carburante

Se observa que las ventas se mantienen estables siempre que el galón de gasolina no supere los 4.25 USD

In [ ]:
plt.figure (figsize = (15,7))
sns.set_style('darkgrid')
sns.set_palette('Paired')
sns.scatterplot ( data = train_df, x = 'Fuel_Price', y =  'Weekly_Sales', hue = 'IsHoliday');

In [ ]:
plt.figure (figsize = (15,7))
sns.set_style('darkgrid')
sns.set_palette('Paired')
sns.scatterplot ( data = train_df, x = 'Fuel_Price', y =  'Weekly_Sales', hue = 'IsHoliday');

#### 2.3.4 Contraste entre Ventas Semanales e IPC

El IPC no parece ser un factor determinante que afecte a las ventas semanales. 

In [ ]:
plt.figure (figsize = (15,7))
sns.set_style('darkgrid')
sns.scatterplot ( data = train_df, x = 'CPI', y =  'Weekly_Sales', hue = 'IsHoliday');

In [ ]:
plt.figure (figsize = (15,7))
sns.set_style('darkgrid')
sns.scatterplot ( data = train_df, x = 'CPI', y =  'Weekly_Sales', hue = 'Type' );

#### 2.3.4 Contraste entre Ventas Semanales y desempleo

In [ ]:
plt.figure (figsize = (15,7))
sns.set_style('darkgrid')
sns.scatterplot ( data = train_df, x = 'Unemployment', y =  'Weekly_Sales', hue = 'IsHoliday' );


In [ ]:
plt.figure (figsize = (15,7))
sns.set_style('darkgrid')
sns.scatterplot ( data = train_df, x = 'Unemployment', y =  'Weekly_Sales', hue = 'Type' );

### 2.4 Matriz de correlación

#### 2.4.1 Matriz de correlación de las diferentes variables


Las rebajas Markdown1 y Markdown4 están sincronizadas para incrementar las ventas en el mes de enero y por eso se observa una correlación del 82%

Las rebajas Markdown3 tienen una lógica correlación positiva con IsHoliday por ser una promoción de Dia de Acción de Gracias.

El precio del combustible tiene una fuerte correlación positiva con el año, confirmando una subida gradual del precio de carburante a lo largo del tiempo.


In [ ]:
sns.set(style="white")

corr = train_df.corr()

mask = np.triu(np.ones_like(corr, dtype=np.bool))

f, ax = plt.subplots(figsize=(20, 15))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

plt.title('Correlation Matrix', fontsize=18)

sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, fmt='.2f')

plt.show()

#### 2.4.2 Tabla de correlación de las diferentes variables vs Ventas Semanales


Las variables que más importancia tienen en las ventas semanales son la superficie de venta de la tienda, el departamento y las diferentes rebajas.

El formato de tienda, el desempleo y el IPC parecen afectar negativamente de forma leve en las ventas cuando estas variables incrementan su valor.


In [ ]:
weekly_sales_corr = train_df.corr().iloc[2,:]

In [ ]:
corr_df = pd.DataFrame(data = weekly_sales_corr, index = weekly_sales_corr.index ).sort_values (by = 'Weekly_Sales', ascending = False)

In [ ]:
plt.figure(figsize = (20,5))
sns.set_palette('mako')
plt.xticks(rotation = 45)
plt.title('Feature correlation')
sns.barplot (data = corr_df, x = corr_df.index, y = 'Weekly_Sales');

## 3. Feature Engineering

<img src=https://image.shutterstock.com/image-vector/vector-background-featuring-abstract-industry-600w-1171013590.jpg>

#### ¿Qué haremos en esta sección?

Crearemos las siguientes variables:

- Semanas clave:
    - SuperBowlWeek
    - LaborDay
    - Thanksgiving
    - Christmas
    
- Variable Quarter
- Variable MarkdownSum
- Rellenar valores missing:
- Encoding categorical data en IsHoliday.
- Encoding categorical data en Type.
- Feature importance
- Análisis de VIF y eliminación de algunas variables.

 


In [ ]:
data_train = train_df.copy()
data_test = test_df.copy()

### 3.1 Feature Engineering Nueva variable:  Semanas Clave


Se van a crear cuatro nuevas variables que corresponden a aquellas semanas que se consideran claves en las ventas, estas serían:

* SuperBowlWeek: Semana 6 del año.
* LaborDay: Semana 36 del año.
* Thanksgiving: Semana 47 del año.
* Christmas: Semana 52 del año.




In [ ]:
data_train['SuperBowlWeek'] = train_df['Week'].apply(lambda x: 1 if x == 6 else 0)
data_train['LaborDay'] = train_df['Week'].apply(lambda x: 1 if x == 36 else 0)
data_train['Thanksgiving'] = train_df['Week'].apply(lambda x: 1 if x == 47 else 0)
data_train['Christmas'] = train_df['Week'].apply(lambda x: 1 if x == 52 else 0)


In [ ]:
data_test['SuperBowlWeek'] = test_df['Week'].apply(lambda x: 1 if x == 6 else 0)
data_test['LaborDay'] = test_df['Week'].apply(lambda x: 1 if x == 36 else 0)
data_test['Thanksgiving'] = test_df['Week'].apply(lambda x: 1 if x == 47 else 0)
data_test['Christmas'] = test_df['Week'].apply(lambda x: 1 if x == 52 else 0)

###  3.2 Feature Engineering nueva variable: Quarter


Agrupamos las semanas por trimestre Q1, Q2,Q3 y Q4. El motivo es para conocer si las ventas y las promociones estan sujetas a los resultados trimestrales que pueda ofrecer la empresa.

* Q1:Semana de la 1 a la 13.
* Q2:Semana de la 14 a la 26.
* Q3:Semana de la 27 a la 39.
* Q4:Semana de la 40 a la 52.


In [ ]:
data_train.insert(24,'Quarter',data_train['Week'])
data_test.insert(23,'Quarter',data_test['Week'])

In [ ]:
data_train['Quarter'] = data_train['Quarter'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13], 1)
data_train['Quarter'] = data_train['Quarter'].replace([14,15,16,17,18,19,20,21,22,23,24,25,26], 2)
data_train['Quarter'] = data_train['Quarter'].replace([27,28,29,30,31,32,33,34,35,36,37,38,39], 3)
data_train['Quarter'] = data_train['Quarter'].replace([40,41,42,43,44,45,46,47,48,49,50,51,52], 4)

data_test['Quarter'] = data_train['Quarter'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13], 1)
data_test['Quarter'] = data_train['Quarter'].replace([14,15,16,17,18,19,20,21,22,23,24,25,26], 2)
data_test['Quarter'] = data_train['Quarter'].replace([27,28,29,30,31,32,33,34,35,36,37,38,39], 3)
data_test['Quarter'] = data_train['Quarter'].replace([40,41,42,43,44,45,46,47,48,49,50,51,52], 4)


### 3.3 Feature Engineering nueva variable: Rebajas


Creamos una nueva variable 'MarkdownSum' que será el sumatorio de las cuatro promociones Markdown. En caso de que 2 promociones se solapen en el tiempo se verá reflejado en el sumatorio.




In [ ]:
data_train['MarkdownsSum'] = train_df['MarkDown1'] + train_df['MarkDown2'] + train_df['MarkDown3'] + train_df['MarkDown4'] + train_df['MarkDown5']
data_test['MarkdownsSum'] = test_df['MarkDown1'] + test_df['MarkDown2'] + test_df['MarkDown3'] + test_df['MarkDown4'] + test_df['MarkDown5']

### 3.3 Feature Engineering: Rellenar valores missing

Podemos comprobar en los gráficos adjuntos la ausencia de bastantes valores en las variables de las rebajas, CPI y Unemployment.

Las rebajas cuando sean NA el valor será sustituido por 0, CPI y Unemployment serán rellenos por la media.


#### 3.3.1 Observamos los valores faltantes

In [ ]:
#Package called missingno (https://github.com/ResidentMario/missingno) !pip install quilt
import missingno as msno
msno.matrix(data_train)

In [ ]:
msno.matrix(data_test)


In [ ]:
# Sumatorio de valores missing para el dataset 'train'.
print('Datos missing en data_train')
print('---------------------')
print(data_train.isna().sum())
print('')
print('Datos missing en data_test')
print('---------------------')
print(data_test.isna().sum())

#### 3.3.2 Los valores missing en 'data_train' los rellenamos con el valor '0'.

Para el caso de 'data_train' solo se observan valores missing en las variables Markdown.

In [ ]:
data_train.fillna(0, inplace = True)

#### 3.3.3 Los valores missing en 'data_test' correspondientes a 'CPI' y 'Unemployment' los rellenamos con la media.

In [ ]:
data_test['CPI'].fillna(data_test['CPI'].mean(), inplace = True)
data_test['Unemployment'].fillna(data_test['Unemployment'].mean(), inplace = True)

In [ ]:
# Los valores missing en 'test' que corresponden solamente a las rebajas los rellenamos con "0"
data_test.fillna(0, inplace = True)

### 3.4 Feature Engineering: IsHoliday Encoding Categorical Data

La variable 'IsHoliday' tendrá valor 1 si su valor original es 'True' y 'False' en caso contrario.



In [ ]:
data_train['IsHoliday'] = data_train['IsHoliday'].apply(lambda x: 1 if x == True else 0)
data_test['IsHoliday'] = data_test['IsHoliday'].apply(lambda x: 1 if x == True else 0)

###  3.5 Feature Engineering : Type  Encoding Categorical Data.

la variable 'Type' tendra valor 1 si la tienda era de la categoria A, 2 si es de la categoria B y 3 si es de la categoria C.

In [ ]:
data_train['Type'] = data_train['Type'].apply(lambda x: 1 if x == 'A' else (2 if x == 'B' else 3))
data_test['Type'] = data_test['Type'].apply(lambda x: 1 if x == 'A' else (2 if x == 'B' else 3))

Resumen:

* Hemos creado nuevas variables en aquellas semanas que son claves en las ventas como SuperBowlWeek, LaborDay, Thanksgiving y Christmas

* Igualmente se crea la variable Quarter en la que agrupamos segun los difereentes Q1,Q2,Q3 y Q4.
* La nueva variable MarkdownSum muestra la suma de todas las promociones para cada fecha dada.
* Se han rellenado los valores missing en data_train y data_test.
* Por ultimo las variables IsHoliday y Type se les aplica Encoding Categorical Data para volverlas numericas.


### 3.6 Feature importance.

#### Selección de variables

Una vez rellenos los valores missing observamos que las variables que poseen más correlación con las ventas semanales son el tamaño de la tienda, el departamento, así como las diferentes campañas de rebajas.


In [ ]:
data_train.corr()['Weekly_Sales'][:5].sort_values(ascending = False)

In [ ]:
X_f = data_train.drop(['Date','Weekly_Sales'], axis = 'columns' )
y_f = data_train['Weekly_Sales']

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_features = RandomForestRegressor() 

In [ ]:
%%time
rf_features.fit(X_f, y_f)

In [ ]:
importance_df = pd.DataFrame({
    'feature': X_f.columns,
    'importance': rf_features.feature_importances_
}).sort_values('importance', ascending=False)

In [ ]:
plt.figure(figsize=(10,6))
plt.title('Feature Importance')
sns.barplot(data=importance_df.head(26), x='importance', y='feature');

In [ ]:
from sklearn.linear_model import LinearRegression

dt_vif = data_train.copy(deep = True)
features = list(dt_vif.columns)
features.remove('Weekly_Sales') # Borrado de la variable objetivo
features.remove('Date') # Una variable en formato Date no puede ser procesada por la funcion.
dt_vif = dt_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = dt_vif[fet]
    y = data_train[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print ('El valor del VIF para la variable', var, 'es:', vif)

Se observan algunas variables que tienen un VIF superior a 5 y algunos valores son muy elevados, procederemos a irlas eliminando y ejecutaremos el proceso nuevamente

In [ ]:
dt_vif = data_train.copy(deep = True)
features = list(dt_vif.columns)
features.remove('Weekly_Sales')
features.remove('Date')
features.remove('MarkdownsSum') # Eliminamos esta variable por ser la que posee mas VIF del conjunto Markdown
features.remove('Week') # Eliminamos la variable Week por tener un VIF de 40,090
dt_vif = dt_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = dt_vif[fet]
    y = data_train[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print ('El valor del VIF para la variable', var, 'es:', vif)

In [ ]:
dt_vif = data_train.copy(deep = True)
features = list(dt_vif.columns)
features.remove('Weekly_Sales')
features.remove('Date')
features.remove('MarkdownsSum') 
features.remove('Week') 
features.remove('Quarter') # Esta variable es la siguientye candidata por tener un VIF de 15


dt_vif = dt_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = dt_vif[fet]
    y = data_train[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print ('El valor del VIF para la variable', var, 'es:', vif)


Existen algunas variables que presentan valor infinito por tanto eliminamos la siguiente variable
   * Eliminar la variable IsHoliday
   

In [ ]:
dt_vif = data_train.copy(deep = True)
features = list(dt_vif.columns)
features.remove('Weekly_Sales')
features.remove('Date')
features.remove('MarkdownsSum') 
features.remove('Week') 
features.remove('Quarter') 
features.remove('IsHoliday') # Eliminamos IsHoliday por tener valor Inf.


dt_vif = dt_vif[features]

for i in range(len(features)):
    var = features[i]
    fet = features[:]
    fet.remove(var)
    
    x = dt_vif[fet]
    y = data_train[var]
    
    model = LinearRegression()
    model.fit(x, y)
    
    vif = 1 / (1 - model.score(x, y))
    
    print ('El valor del VIF para la variable', var, 'es:', vif)

    

## 4. Machine Learning

<img src=https://image.shutterstock.com/image-vector/big-data-artificial-intelligence-concept-600w-1092234560.jpg>

Empezaremos por eliminar de los dataset data_train y data_test las siguientes variables :

'MarkdownsSum', 'Week', 'Quarter', 'IsHoliday' y 'Date'

In [ ]:
data_train.drop(['MarkdownsSum', 'Week','Quarter','IsHoliday','Date'], axis = 'columns', inplace=True)
data_test.drop(['MarkdownsSum', 'Week','Quarter','IsHoliday','Date'], axis = 'columns', inplace=True)

In [ ]:
print(data_train.columns)
print('---------------------------------------------------------------------------')
print(data_test.columns)


In [ ]:
X = data_train[['Store', 'Dept','Temperature', 'Fuel_Price',
       'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI',
       'Unemployment', 'Type', 'Size', 'Day', 'Month', 'Year', 'SuperBowlWeek',
       'LaborDay', 'Thanksgiving', 'Christmas']]

y = data_train['Weekly_Sales']

Utilizaremos train_test_split y dividiremos el conjunto en 70% training y el 30% en test.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### 4.1 LinearRegression

Regresión lineal por mínimos cuadrados ordinarios.

LinearRegression se ajusta a un modelo lineal con coeficientes w = (w1,…, wp) para minimizar la suma residual de cuadrados entre los objetivos observados en el conjunto de datos y los objetivos predichos por la aproximación lineal.

###### Kaggle Score : 19,651


In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

# Creamos modelo de Regresion Lineal
lrModel = lr.fit(X_train, y_train) 

# Prediccion para Train
preds_train_LR = lrModel.predict(X_train) 
# Prediccion para Test
preds_test_LR = lrModel.predict(X_test) 

print('R2 medio para Train es : ',metrics.mean_squared_error(y_train, preds_train_LR, squared = False)) # Obtenemos el R2 medio para Train
print('R2 medio para Test es : ',metrics.mean_squared_error(y_test, preds_test_LR , squared = False)) #Obtenemos el R2 medio para Test



In [ ]:
test = data_test[['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type',
       'Size', 'Day', 'Month', 'Year', 'SuperBowlWeek', 'LaborDay',
       'Thanksgiving', 'Christmas']]

predict_LR = lrModel.predict(test)

In [ ]:
sampleSubmission['Weekly_Sales'] = predict_LR
sampleSubmission.to_csv('submission_LR.csv',index=False)

In [ ]:
sampleSubmission.head()


### 4.2 GradientBoostingRegressor

GB construye un modelo aditivo de manera progresiva por etapas; permite la optimización de funciones de pérdida diferenciables arbitrarias. En cada etapa se ajusta un árbol de regresión al gradiente negativo de la función de pérdida dada.

##### Kaggle Score : 10,038

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(random_state=42)

# Creamos modelo Gradient Boosting Regressor
gbModel = gb.fit(X_train, y_train)

# Prediccion para Train
preds_train_GBR = gbModel.predict(X_train)
# Prediccion para Test
preds_test_GBR = gbModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ',metrics.mean_squared_error(y_train, preds_train_GBR, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ',metrics.mean_squared_error(y_test, preds_test_GBR, squared = False))


In [ ]:
test = data_test[['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type',
       'Size', 'Day', 'Month', 'Year', 'SuperBowlWeek', 'LaborDay',
       'Thanksgiving', 'Christmas']]
predict_GBR = gb.predict(test)

In [ ]:
sampleSubmission['Weekly_Sales'] = predict_GBR
sampleSubmission.to_csv('submission_GBR.csv',index=False)

In [ ]:
sampleSubmission.head()

### 4.3 XGBRegressor

XGBoost significa "Extreme Gradient Boosting" y es una implementación del algoritmo de árboles de aumento de gradiente. XGBoost es un popular modelo de aprendizaje automático supervisado con características como velocidad de cálculo, paralelización y rendimiento.

##### Kaggle Score : 5,545

In [ ]:
from xgboost import XGBRegressor

gbm = XGBRegressor(random_state=42, n_jobs=-1)

# Creamos modelo Gradient Boosting Regressor
gbmModel = gbm.fit(X_train, y_train)

# Prediccion para Train
preds_train_XGB = gbmModel.predict(X_train)
# Prediccion para Test
preds_test_XGB = gbmModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ',metrics.mean_squared_error(y_train, preds_train_XGB, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ',metrics.mean_squared_error(y_test, preds_test_XGB, squared = False))



In [ ]:
test = data_test[['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type',
       'Size', 'Day', 'Month', 'Year', 'SuperBowlWeek', 'LaborDay',
       'Thanksgiving', 'Christmas']]
predict_XGB = gbm.predict(test)

In [ ]:
sampleSubmission['Weekly_Sales'] = predict_XGB
sampleSubmission.to_csv('submission_XGB.csv',index=False)

In [ ]:
sampleSubmission.head()

### 4.4 RandomForestRegressor

Un RandomForestRegressor es un metaestimador que se ajusta a una serie de árboles de decisión de clasificación en varias submuestras del conjunto de datos y utiliza promedios para mejorar la precisión predictiva y controlar el sobreajuste. El tamaño de la submuestra se controla con el parámetro max_samples si bootstrap = True (predeterminado); de lo contrario, se usa todo el conjunto de datos para construir cada árbol.

 - n_estimatorsint, default=100
 - criterion{“mse”, “mae”}, default=”mse”
 - max_depth: int, default=None
 - min_samples_split: int or float, default=2
 - min_samples_leaf: int or float, default=1
 - min_weight_fraction_leaf: float, default=0.0
 - max_features: {“auto”, “sqrt”, “log2”}, int or float, default=”auto”
 - max_leaf_nodes: int, default=None
 - min_impurity_decrease: float, default=0.0
 - min_impurity_split: float, default=None
 - bootstrap: bool, default=True
 - oob_score: bool, default=False
 - n_jobs: int, default=None




#### 4.4.1 RandomForestRegressor parametros de serie
##### Kaggle Score : 3,693

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

#Creamos modelo  Random Forest Regressor
rfModel = rf.fit(X_train, y_train)

# Prediccion para Train
preds_train_RFR = rfModel.predict(X_train)
# Prediccion para Test
preds_test_RFR = rfModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ', metrics.mean_squared_error(y_train, preds_train_RFR, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ', metrics.mean_squared_error(y_test, preds_test_RFR, squared = False))




In [ ]:
test = data_test[['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type',
       'Size', 'Day', 'Month', 'Year', 'SuperBowlWeek', 'LaborDay',
       'Thanksgiving', 'Christmas']]
predict_RFR = rf.predict(test)

In [ ]:
sampleSubmission['Weekly_Sales'] = predict_RFR
sampleSubmission.to_csv('submission_RFR.csv',index=False)

In [ ]:
sampleSubmission.head()

#### 4.4.2 RandomForestRegressor con bootstrap = False

Vamos a cambiar el parametro de bootstrap a False, con esto conseguiremos utilizar todas las variables en cada arbol.
Se comprueba que da mal resultado pues existe overfitting en Train con un R2 medio muy bajo pero en Test es mas impreciso que en la version original.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, max_depth=None,random_state=42,bootstrap=False)

#Creamos modelo  Random Forest Regressor
rfModel = rf.fit(X_train, y_train)

# Prediccion para Train
preds_train_RFR = rfModel.predict(X_train)
# Prediccion para Test
preds_test_RFR = rfModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ', metrics.mean_squared_error(y_train, preds_train_RFR, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ', metrics.mean_squared_error(y_test, preds_test_RFR, squared = False))

#### 4.4.3 RandomForestRegressor con max_depth = 30
 Probamos limitando la profundidad del arbol por si de esta manera evitamos que se produzca un sobreajuste.
 
Se ha probado con diferentes ajustes pero con valores de 15 o 20 se ha comprobado que el R2 en Train y Test empeora notablemente.
Se observa una muy leve mejora pero es un punto de partida para ir cambiando otros parametros

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=30, random_state=42)

#Creamos modelo  Random Forest Regressor
rfModel = rf.fit(X_train, y_train)

# Prediccion para Train
preds_train_RFR = rfModel.predict(X_train)
# Prediccion para Test
preds_test_RFR = rfModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ', metrics.mean_squared_error(y_train, preds_train_RFR, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ', metrics.mean_squared_error(y_test, preds_test_RFR, squared = False))

#### 4.4.4 RandomForestRegressor con max_depth = 30 + max features = 4,6,10,15

Vamos a limitar el numero de variables en cada split para comprobar si de esa manera conseguimos una mejora en el modelo anterior.

Con un numero de 4,6, y 10 hemos empeorado el modelo, por tanto ponemos el limite en 15.

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=30, random_state=42,max_features=15)

#Creamos modelo  Random Forest Regressor
rfModel = rf.fit(X_train, y_train)

# Prediccion para Train
preds_train_RFR = rfModel.predict(X_train)
# Prediccion para Test
preds_test_RFR = rfModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ', metrics.mean_squared_error(y_train, preds_train_RFR, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ', metrics.mean_squared_error(y_test, preds_test_RFR, squared = False))

#### 4.4.5 RandomForestRegressor con max_depth = 30 + max features = 15, n_jobs = -1

Conseguimos una mejora en tiempo de respuesta, al obtener resultados en 23 segundos frente a los 4 minutos de la version inicial.
No obstante los resultados del modelo en Train y Test no varian frente a la version anterior

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=30, random_state=42,max_features=15,n_jobs=-1 )

#Creamos modelo  Random Forest Regressor
rfModel = rf.fit(X_train, y_train)

# Prediccion para Train
preds_train_RFR = rfModel.predict(X_train)
# Prediccion para Test
preds_test_RFR = rfModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ', metrics.mean_squared_error(y_train, preds_train_RFR, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ', metrics.mean_squared_error(y_test, preds_test_RFR, squared = False))

#### 4.4.6 RandomForestRegressor con max_depth = 30 + max features = 15, n_jobs = -1, n_estimators = 80

Limitando el numero de arboles a 80 obtenemos una ligera mejora. Por debajo de 80 y por encima de este valor hemos obtenido resultados peores.

Hemos conseguido una leve mejora en la prediccion frente a la version sin modificar parametros.

#### Kaggle Score :3,569

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=80 ,max_depth=30, random_state=42,max_features=15,n_jobs=-1)

#Creamos modelo  Random Forest Regressor
rfModel = rf.fit(X_train, y_train)

# Prediccion para Train
preds_train_RFR = rfModel.predict(X_train)
# Prediccion para Test
preds_test_RFR = rfModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ', metrics.mean_squared_error(y_train, preds_train_RFR, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ', metrics.mean_squared_error(y_test, preds_test_RFR, squared = False))

In [ ]:
test = data_test[['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type',
       'Size', 'Day', 'Month', 'Year', 'SuperBowlWeek', 'LaborDay',
       'Thanksgiving', 'Christmas']]
predict_RFR_Tunned = rf.predict(test)

In [ ]:
sampleSubmission['Weekly_Sales'] = predict_RFR_Tunned
sampleSubmission.to_csv('submission_RFR_Tunned.csv',index=False)

#### 4.4.7 RandomForestRegressor con max_depth = 30 + max features = 15, n_jobs = -1, n_estimators = 80, min_samples_split = 4

limitamos aun mas las capacidades de cada arbol, en este caso el numero minimo de ejemplos requeridos para que se expanda un nodo y cree ramas pasara de 2 a 4. No se ha encontrado un empeoramiento significativo por lo que lo mantendremos de esta manera.

Finalmente comprobamos la prediccion en Kaggle y observamos que empeora por lo que este cambio no ha sido beneficioso.

#### Kaggle Score: 3,613

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=80 ,min_samples_split = 4, max_depth=30, random_state=42,max_features=15,n_jobs=-1)

#Creamos modelo  Random Forest Regressor
rfModel = rf.fit(X_train, y_train)

# Prediccion para Train
preds_train_RFR = rfModel.predict(X_train)
# Prediccion para Test
preds_test_RFR = rfModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ', metrics.mean_squared_error(y_train, preds_train_RFR, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ', metrics.mean_squared_error(y_test, preds_test_RFR, squared = False))

In [ ]:
test = data_test[['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type',
       'Size', 'Day', 'Month', 'Year', 'SuperBowlWeek', 'LaborDay',
       'Thanksgiving', 'Christmas']]
predict_RFR_Tunned2 = rf.predict(test)

In [ ]:
sampleSubmission['Weekly_Sales'] = predict_RFR_Tunned2
sampleSubmission.to_csv('submission_RFR_Tunned2.csv',index=False)

#### 4.4.8 RandomForestRegressor con max_depth = 30 + max features = 15, n_jobs = -1, n_estimators = 80, , min_samples_leaf= 2

Por ultimo, ajustaremos el algoritmo para que el numero minimo de muestras necesarias para estar en un nodo de hoja pase de 1 a 2.

Se comprueba que perjudica a Train pero no tanto a Test y nos podria dar una pista que nos indica que evita el sobreajuste.

Finalmente comprobamos que esta opcion tampoco mejora el modelo.

#### Kaggle Score: 3,594

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=80,min_samples_leaf= 2, max_depth=30, random_state=42,max_features=15,n_jobs=-1)

#Creamos modelo  Random Forest Regressor
rfModel = rf.fit(X_train, y_train)

# Prediccion para Train
preds_train_RFR = rfModel.predict(X_train)
# Prediccion para Test
preds_test_RFR = rfModel.predict(X_test)

# Obtenemos el R2 medio para Train
print('R2 medio para Train es : ', metrics.mean_squared_error(y_train, preds_train_RFR, squared = False))
# Obtenemos el R2 medio para Test
print('R2 medio para Test es : ', metrics.mean_squared_error(y_test, preds_test_RFR, squared = False))

In [ ]:
test = data_test[['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type',
       'Size', 'Day', 'Month', 'Year', 'SuperBowlWeek', 'LaborDay',
       'Thanksgiving', 'Christmas']]
predict_RFR_Tunned3 = rf.predict(test)

In [ ]:
sampleSubmission['Weekly_Sales'] = predict_RFR_Tunned3
sampleSubmission.to_csv('submission_RFR_Tunned3.csv',index=False)